<h2>Import Packages</h2>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h2>Get source code from Wikipedia URL</h2>

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

<h2>Get source code for ONLY the table of interest in the Wikipedia article</h2>
<p>Row 1: source code for the table</p>
<p>Row 2: source doe for the rows in the table</p>

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
table_rows = My_table.find_all('tr')

<h2>Initiate Pandas DataFrame from source code</h2>

In [4]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=['Postcode', 'Borough', 'Neighbourhood'])

<h2>Clean up DataFrame</h2>
<p>Rows 1-2: Remove all whitespace characters</p>
<p>Row 3: Drop the first row (which was the header row)</p>
<p>Row 4: Remove all rows where Borough was Not Assigned</p>
<p>Rows 5-7: Loop through rows, and if Neighbourhood was Not Assigned, change the name of the Neighborhood to its corresponding Borough</p>
<p>Rows 8-9: Group rows by Postcodes/Boroughs</p> 

In [5]:
df = df.replace(r'\\n',' ', regex=True)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop([0])
df = df[df.Borough != 'Not assigned']
for row in range(len(df)):
    if df.iloc[row,2] == 'Not assigned':
        df.iloc[row,2] = df.iloc[row,1]
df["Neighbourhood"] = df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
df = df.drop_duplicates()
df

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Harbourfront, Regent Park"
7,M6A,North York,"Lawrence Heights, Lawrence Manor"
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,"Rouge, Malvern"
15,M3B,North York,Don Mills North
16,M4B,East York,"Woodbine Gardens, Parkview Hill"
18,M5B,Downtown Toronto,"Ryerson, Garden District"


<h2>Show Dimensions of DataFrame (which has 103 rows)</h2>

In [6]:
print(df.shape)

(103, 3)


<h2>Import Geo Data</h2>

In [7]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_b261bf3e12a045ff909b5d2a62b31142 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='OGdvo4s_te15srCGrCbJ7RVfFTUBSLUYuMvCPlp_p7he',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_b261bf3e12a045ff909b5d2a62b31142.get_object(Bucket='courseracapstoneproject-donotdelete-pr-eymr55m1tsxq5b',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geo = pd.read_csv(body)
geo.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2>Merge the two dataframes to connect Latitude and Longitude to the Postal Codes</h2>

In [8]:
df = df.merge(geo, left_on='Postcode', right_on='Postal Code', how='outer')

<h2>Drop redundant column name</h2>

In [9]:
df = df.drop(['Postal Code'], axis=1)

In [10]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


<h2>Show dimensions of dataframe</h2>

In [11]:
df.shape

(103, 5)